In [ ]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [ ]:
import pkg_resources
pkg_resources.get_distribution('open3d').version

In [ ]:
input_path="E:\Downloads/"
output_path="E:\Downloads/"
dataname="XYZpntcloud.xyz"
point_cloud= np.loadtxt(input_path+dataname,delimiter=',',skiprows=1)

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud[:,:3])
#pcd.colors = o3d.utility.Vector3dVector(point_cloud[:,3:6]/255)
#pcd.normals = o3d.utility.Vector3dVector(point_cloud[:,6:9])

In [ ]:
o3d.visualization.draw_geometries([pcd])


In [ ]:
#point cloud metrics

np.mean(np.asarray(pcd.points),axis=0)[1]

In [ ]:
#matplot
mean_Z=np.mean(pcd.points,axis=0)[2]
spatial_query=np.asarray(pcd.points)[abs( np.asarray(pcd.points)[:,2]-mean_Z)<1]
xyz=spatial_query[:,:3]


In [ ]:
ax = plt.axes(projection='3d')
ax.scatter(xyz[:,0], xyz[:,1], xyz[:,2],  s=0.1)


In [ ]:
#computing Normals
print("Recompute the normal of the downsampled point cloud")
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=30))
print("done")

In [ ]:
#orient on plane
#breaks the data somehow
pcd.orient_normals_consistent_tangent_plane(10)
print('done')

In [ ]:
o3d.visualization.draw_geometries([pcd],point_show_normal=True)

In [ ]:
#POISSON APPROACH

In [ ]:
poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9, width=0, scale=1.2, linear_fit=True)[0]
print("done")

In [ ]:
o3d.visualization.draw_geometries([poisson_mesh])


In [ ]:
#BALL PIVOTING

In [ ]:
#the mesh breaks the visualisation tool somehow
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist

bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 2]))


In [ ]:
dec_mesh = bpa_mesh.simplify_quadric_decimation(100000)
dec_mesh.remove_degenerate_triangles()
dec_mesh.remove_duplicated_triangles()
dec_mesh.remove_duplicated_vertices()
dec_mesh.remove_non_manifold_edges()

In [ ]:
#o3d.visualization.draw_geometries([bpa_mesh])


In [ ]:
#CONVEX HULL